### Convert raw data csv file to Structured Dataset csv file

In [173]:
import pandas
import sys
import numpy

bacteria_name = "Klebsiella pneumoniae"
bacteria_short_name = "KP"
antibiotic_name = "meropenem"

file_to_read = './raw_data_from_NCBI/' + antibiotic_name + '.csv'
raw_data = pandas.read_csv(file_to_read)



In [174]:
# Dropping unwanted column
unwanted_columns = ["Strain", "Isolate identifiers", "AMR genotypes core"]
raw_data.drop(labels=unwanted_columns, inplace=True, axis=1)
raw_data.head()


,#Organism group,Isolate,AMR genotypes,AST phenotypes
0,Enterobacter,PDT000021125.1,"aac(3)-Ib=COMPLETE,aph(3')-Ia=COMPLETE,blaACT-...",meropenem=ND
1,Enterobacter,PDT000021126.1,"aadA1=COMPLETE,aph(3'')-Ib=COMPLETE,aph(6)-Id=...",meropenem=ND
2,Enterobacter,PDT001009300.1,"aadA1=COMPLETE,aph(3'')-Ib=COMPLETE,aph(6)-Id=...",meropenem=ND
3,Enterobacter,PDT000026740.1,"aac(6')-Ib-cr5=COMPLETE,aph(3')-Ia=COMPLETE,ar...",meropenem=ND
4,Klebsiella pneumoniae,PDT000022765.1,"blaKPC-3=COMPLETE,blaSHV-12=COMPLETE,blaTEM-1=...",meropenem=ND


In [175]:
# Dropping unwanted rows
raw_data.drop(raw_data[(raw_data['#Organism group'] != bacteria_name )].index, inplace=True)
copy_raw_data = raw_data
for index in raw_data.index:
    if (raw_data['AST phenotypes'][index]).split(',').count("meropenem=ND") > 0:
        copy_raw_data = copy_raw_data.drop(index)
    elif (raw_data['AST phenotypes'][index]).split(',').count("meropenem=I") > 0:
        copy_raw_data = copy_raw_data.drop(index)
    elif (raw_data['AST phenotypes'][index]).split(',').count("meropenem=R") > 0:
        copy_raw_data['AST phenotypes'][index] = "meropenem=R"
    elif (raw_data['AST phenotypes'][index]).split(',').count("meropenem=S") > 0:    
        copy_raw_data['AST phenotypes'][index] = "meropenem=S"

raw_data = copy_raw_data.reset_index(drop=True)
raw_data.head()


,#Organism group,Isolate,AMR genotypes,AST phenotypes
0,Klebsiella pneumoniae,PDT001161721.1,"aac(3)-IId=COMPLETE,aac(3)-IIe=COMPLETE,aac(6'...",meropenem=R
1,Klebsiella pneumoniae,PDT001161722.1,"aac(3)-IId=COMPLETE,aadA5=COMPLETE,aph(3'')-Ib...",meropenem=S
2,Klebsiella pneumoniae,PDT000595118.1,"aac(6')-Ib-cr5=COMPLETE,aph(3'')-Ib=COMPLETE,a...",meropenem=R
3,Klebsiella pneumoniae,PDT000595124.1,"aac(6')-Ib-cr5=COMPLETE,aph(3'')-Ib=COMPLETE,a...",meropenem=R
4,Klebsiella pneumoniae,PDT000595117.1,"aac(3)-IIe=COMPLETE,aac(6')-Ib-cr5=COMPLETE,ap...",meropenem=R


In [176]:
columns = ["Isolate"]
structured_data = pandas.DataFrame(columns=columns)

for index in raw_data.index:
    structured_data.loc[index] = [raw_data["Isolate"][index]]        

for index in raw_data.index:
    for gene in (raw_data["AMR genotypes"][index]).split(","):
        if gene in structured_data.columns:
            structured_data[gene][index] = 1
        else:
            structured_data[gene] = 0
            structured_data[gene][index] = 1

# column_to_move = structured_data.pop("AST phenotypes")
# structured_data.insert(len(structured_data),"AST phenotypes", column_to_move)

structured_data["AST phenotypes"] = raw_data["AST phenotypes"]
display(structured_data)


/tmp/ipykernel_19099/578140448.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_data[gene][index] = 1
/tmp/ipykernel_19099/578140448.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_data[gene][index] = 1
/tmp/ipykernel_19099/578140448.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_data[gene][index] = 1
/tmp/ipykernel_19099/578140448.py:13: SettingWithCopyWarning: 
A value is trying to be s

,Isolate,aac(3)-IId=COMPLETE,aac(3)-IIe=COMPLETE,aac(6')-Ib-cr5=COMPLETE,aadA1=COMPLETE,aadA2=COMPLETE,aph(3'')-Ib=COMPLETE,aph(3')-Ia=COMPLETE,aph(6)-Id=COMPLETE,arr-2=COMPLETE,...,aph(3')-II=HMM,ble=HMM,blaSHV-202=COMPLETE,blaSHV-171=COMPLETE,rmtH=COMPLETE,oqxR_Q11L=POINT,blaSHV-142=COMPLETE,mcr-1.1=COMPLETE,blaSHV-200=COMPLETE,AST phenotypes
0,PDT001161721.1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,meropenem=R
1,PDT001161722.1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=S
2,PDT000595118.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=R
3,PDT000595124.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=R
4,PDT000595117.1,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,PDT001128534.2,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=R
678,PDT001128416.2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,meropenem=S
679,PDT001185294.1,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=S
680,PDT000326682.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,meropenem=S


In [178]:
for index in structured_data.index:
    if structured_data["AST phenotypes"][index] == "meropenem=R":
        structured_data["AST phenotypes"][index] = 1
    
    if structured_data["AST phenotypes"][index] == "meropenem=S":
        structured_data["AST phenotypes"][index] = 0

display(structured_data)

/tmp/ipykernel_19099/1385450214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_data["AST phenotypes"][index] = 1
/tmp/ipykernel_19099/1385450214.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_data["AST phenotypes"][index] = 0


,Isolate,aac(3)-IId=COMPLETE,aac(3)-IIe=COMPLETE,aac(6')-Ib-cr5=COMPLETE,aadA1=COMPLETE,aadA2=COMPLETE,aph(3'')-Ib=COMPLETE,aph(3')-Ia=COMPLETE,aph(6)-Id=COMPLETE,arr-2=COMPLETE,...,aph(3')-II=HMM,ble=HMM,blaSHV-202=COMPLETE,blaSHV-171=COMPLETE,rmtH=COMPLETE,oqxR_Q11L=POINT,blaSHV-142=COMPLETE,mcr-1.1=COMPLETE,blaSHV-200=COMPLETE,AST phenotypes
0,PDT001161721.1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,PDT001161722.1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,PDT000595118.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,PDT000595124.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,PDT000595117.1,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,PDT001128534.2,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
678,PDT001128416.2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
679,PDT001185294.1,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
680,PDT000326682.1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
file_path = "./datasets/"
file_name = f"{bacteria_short_name}_{antibiotic_name}.csv"
structured_data.to_csv(file_path+file_name)